In [5]:
import pandas as pd


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

column_names = [
    "hotel_name", "city", "country", "price_per_night_text", "price_per_night", "rating", "review_count_text", "review_count", "stars", "location", "amenities"
]

df = pd.read_csv("vietnam_hotels_data.csv", names=column_names, na_values="?")

df.head(221)

,hotel_name,city,country,price_per_night_text,price_per_night,rating,review_count_text,review_count,stars,location,amenities
0,hotel_name,city,country,price_per_night_text,price_per_night,rating,review_count_text,review_count,stars,location,amenities
1,Bay Hotel Ho Chi Minh,Ho Chi Minh,Vietnam,"₫ 2,238,176",2238176,8.3 Excellent,"4,634 reviews",4634.0,4.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Free fitness center access', '+10']"
2,LOTTE HOTEL SAIGON,Ho Chi Minh,Vietnam,"₫ 4,572,727",4572727,9.0 Exceptional,"9,953 reviews",9953.0,5.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Free fitness center access', '+8']"
3,Belvilla Comfort Stays Near SECC formerly Cozy Hotel,Ho Chi Minh,Vietnam,"₫ 424,252",424252,8.8 Excellent,404 reviews,404.0,4.0,"District 7, Ho Chi Minh City - 3.5 km to center",[]
4,Yoko Airport Saigon Hotel,Ho Chi Minh,Vietnam,"₫ 357,736",357736,8.0 Excellent,"1,464 reviews",1464.0,4.0,"Tân Bình, Ho Chi Minh City - 4.7 km to center","['Parking', 'Express check-in', 'Free WiFi']"
5,Asian Ruby Center Point Hotel,Ho Chi Minh,Vietnam,"₫ 448,242",448242,7.6 Very good,"2,938 reviews",2938.0,3.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Welcome drink', 'Parking', 'Free WiFi']"
6,Khach san Muong Thanh Luxury Saigon Hotel,Ho Chi Minh,Vietnam,"₫ 1,417,392",1417392,8.6 Excellent,"5,659 reviews",5659.0,4.0,"Phú Nhuận, Ho Chi Minh City - 3.2 km to center","['Breakfast', 'Free fitness center access', '+5']"
7,Fusion Suites Saigon,Ho Chi Minh,Vietnam,"₫ 3,120,137",3120137,9.0 Exceptional,"6,638 reviews",6638.0,4.0,"District 1, Ho Chi Minh City - City center",[]
8,Le Vu Hotel,Ho Chi Minh,Vietnam,"₫ 297,778",297778,8.4 Excellent,637 reviews,637.0,3.0,"District 1, Ho Chi Minh City - City center","['Welcome drink', 'Parking', 'Room sanitizing', '+2']"
9,Sunshine Airport Hotel,Ho Chi Minh,Vietnam,"₫ 395,631",395631,8.7 Excellent,70 reviews,70.0,4.0,"Tân Bình, Ho Chi Minh City - 4.6 km to center","['Parking', 'Express check-in', 'Free WiFi']"
